In [1]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
#pandas as pd 其實可以不用啦 啊不知道ㄌ

In [2]:
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
# 大小：三乘三 所以是2D
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import SGD
#可以去搜尋有什麼有名的優化方法

### 1. 讀入 MNSIT 數據集

In [3]:
from tensorflow.keras.datasets import mnist

In [4]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
#training跟testing的資料要分開讀

11493376/11490434 [==============================] - 0s 0us/step


### 2. 資料整理

### Channel

CNN 要注意一張圖有多少個 channel, 開始我們因為只有灰階, 所以只有一個 channel。因此我們要轉一下我們的資料格式:

    (28,28) --> (28, 28, 1)

In [5]:
x_train = x_train.reshape(60000, 28, 28, 1) / 255
#cnn要告訴他有幾個channel 同樣28*28如果是RGB就是三個channel 現在是灰階channel是一個

In [6]:
x_test = x_test.reshape(10000, 28, 28, 1) / 255

In [7]:
y_train = to_categorical(y_train, 10)
#5個種類就改成5
y_test = to_categorical(y_test, 10)

### 3. step 1: 打造函數學習機 (CNN)

來做三層conv吧
第一層做8個filter

In [31]:
model = Sequential()
#打開函數學習機

In [32]:
model.add(Conv2D(8, (3,3), padding='same',
                 input_shape=(28,28,1),
                activation='relu'))
#第一層8個filter 回家作業可以改這個

In [33]:
model.add(MaxPooling2D(pool_size=(2,2)))

In [34]:
model.add(Conv2D(16, (3,3), padding='same',
                activation='relu'))

In [35]:
model.add(MaxPooling2D(pool_size=(2,2)))

In [36]:
model.add(Conv2D(32, (3,3), padding='same',
                activation='relu'))

In [37]:
model.add(MaxPooling2D(pool_size=(2,2)))

In [38]:
model.add(Flatten())

In [39]:
model.add(Dense(20, activation='relu'))
#決定該層使用幾個神經元（可改）

In [40]:
model.add(Dense(10, activation='softmax'))
#前面輸入10個所以這裡只能是10（輸出幾個神經元）

#### 看一下我們的神經網路

In [41]:
model.summary()
#第一行8為記分板個數（之前設定的），28*28為大小，到288拉平
#9個權重+一個偏值=10 阿有8個計分板 10*8=80個要掃

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 28, 28, 8)         80        
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 14, 14, 8)         0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 14, 14, 16)        1168      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 7, 7, 16)          0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 7, 7, 32)          4640      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 3, 3, 32)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 288)              

#### 組裝

In [42]:
model.compile(loss='mse', optimizer=SGD(lr=0.087),
             metrics=['accuracy'])

### 4. step 2: fit

In [43]:
model.fit(x_train, y_train, batch_size=128, epochs=12)
#訓練12次 應該可以改ㄅ

Epoch 1/12
469/469 [==============================] - 1s 3ms/step - loss: 0.0896 - accuracy: 0.1662
Epoch 2/12
469/469 [==============================] - 1s 3ms/step - loss: 0.0891 - accuracy: 0.2196
Epoch 3/12
469/469 [==============================] - 1s 3ms/step - loss: 0.0879 - accuracy: 0.3224
Epoch 4/12
469/469 [==============================] - 2s 3ms/step - loss: 0.0819 - accuracy: 0.4734
Epoch 5/12
469/469 [==============================] - 1s 3ms/step - loss: 0.0441 - accuracy: 0.7240
Epoch 6/12
469/469 [==============================] - 1s 3ms/step - loss: 0.0262 - accuracy: 0.8221
Epoch 7/12
469/469 [==============================] - 1s 3ms/step - loss: 0.0227 - accuracy: 0.8432
Epoch 8/12
469/469 [==============================] - 2s 3ms/step - loss: 0.0210 - accuracy: 0.8531
Epoch 9/12
469/469 [==============================] - 2s 3ms/step - loss: 0.0199 - accuracy: 0.8597
Epoch 10/12
469/469 [==============================] - 1s 3ms/step - loss: 0.0191 - accuracy: 0.8642

回家作業：唯一要求9成五以上就好

### Step 3. 預測

In [44]:
result = model.predict_classes(x_test)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


### 看看測試資料表現如何

In [45]:
loss, acc = model.evaluate(x_test, y_test)

313/313 [==============================] - 1s 2ms/step - loss: 0.0176 - accuracy: 0.8747


In [46]:
print(f'測試資料的正確率為 {acc*100:.2f}%')

測試資料的正確率為 87.47%


In [47]:
def my_predict(n):
    print('我可愛的 CNN 預測是', result[n])
    X = x_test[n].reshape(28,28)
    plt.imshow(X, cmap='Greys')

In [48]:
from ipywidgets import interact_manual

In [49]:
interact_manual(my_predict, n=(0, 9999));

interactive(children=(IntSlider(value=4999, description='n', max=9999), Button(description='Run Interact', sty…

### 把我們的 model 存起來

In [ ]:
model.save('myCNNmodel.h5')